In [1]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt,butter, lfilter
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler


eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()
from eegPinelineDesign import CenterAtPeakOfWindow,Threshold_test,spindle_overlapping_test,used_windows_check,cut_segments

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 'logistic_model.pkl', 'pineline 13.csv', 'pipe line 10', 'pipe line 10.csv', 'pipeline 13.pickle', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'single subject prediction.p', 'single subject testing.p', 'single subject.p', 'Sleep Stage Scoring Criteria.docx', 'spindle training.p', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_an

In [3]:
import pickle
with open('training data.p','rb') as handle:
    result=pickle.load(handle)
    
channelList=['F3','F4','C3','C4','O1','O2']
X = [];Y=[]
for num, EDFfileName in enumerate(EDFfiles):
    if EDFfileName == 'suj13_l2nap_day2 edited.edf' or EDFfileName =='suj13_l2nap_day2 edited1.edf':
        pass
    else:
        file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=num)


        for names in channelList:
            for item in result['spindle'][fileName][names]:
                if item.shape[1] == 3000:

                    X.append(item[0,:])
                    Y.append(1)
            for item in result['non spindle'][fileName][names]:
                if item.shape[1] == 3000:
                    X.append(item[0,:])
                    Y.append(0)

X=np.array(X);Y=np.array(Y)

idx=np.arange(X.shape[0])
GG=np.random.choice(tuple(idx),len(idx),replace=False)
def shuffle(x):
    return sorted(x, key=lambda k: random.random())
GG = shuffle(shuffle(shuffle(shuffle(GG))))
XX=[];YY=[]
for idxx in GG:
    XX.append(X[idxx])
    YY.append(Y[idxx])
    
from sklearn.preprocessing import normalize
normal_X = normalize(XX)

clf =LogisticRegression(penalty='l2',C=.1,tol=10e-9,fit_intercept=True,solver='liblinear',
                                             max_iter=5e8,multi_class='ovr',n_jobs=-1)



clf.fit(normal_X,YY)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500000000.0, multi_class='ovr',
          n_jobs=-1, penalty='l2', random_state=None, solver='liblinear',
          tol=1e-08, verbose=0, warm_start=False)

In [4]:
num=2
file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=num)
channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
raw_filter = eegPinelineDesign.load_data(file_to_read,channelList,11, 16)#spindle pass
raw_alpha=eegPinelineDesign.load_data(file_to_read,channelList,8, 12)#alpha pass
raw_spindle=eegPinelineDesign.load_data(file_to_read,channelList,11, 16)#spindle pass
raw_muscle=eegPinelineDesign.load_data(file_to_read,channelList,30, 40)#
channelList = ['F3','F4','C3','C4','O1','O2']
raw_filter.pick_channels(channelList)

Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    2, 7, 4, 4, 2
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 3 ICA components
Band-pass filtering from 11 - 16 Hz
Extracting edf Parameters from suj10_d1final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3601999  =      0.000 ...  3601.999 secs...
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all P

<RawEDF  |  suj10_d1final.edf, n_channels x n_times : 6 x 3602000 (3602.0 sec)>

In [7]:
annotation_to_read = [x for x in Annotationfiles if fileName in x]
file = pd.read_csv(annotation_to_read[0])
labelFind = re.compile('spindle',eegPinelineDesign.re.IGNORECASE)
spindles=[]# take existed annotations
for row in file.iterrows():
    currentEvent = row[1][-1]
    if labelFind.search(currentEvent):
        spindles.append(row[1][0])# time of marker
print(spindles)

[306.56, 308.603, 309.972, 314.434, 321.003, 337.655, 372.71, 386.15, 442.946, 446.895, 450.7, 458.861, 465.311, 481.595, 542.337, 554.0, 571.428, 624.319, 625.847, 666.74, 675.803, 733.183, 760.274, 764.011, 766.605, 785.35, 792.313, 840.669, 842.973, 847.159, 850.371, 899.037, 908.949, 917.387, 970.358, 988.0055482, 1035.83, 1047.203, 1049.204, 1052.14, 1064.356, 1081.982, 1084.917, 1105.258, 1164.727, 1168.913, 1173.869, 1181.438, 1185.299, 1231.585, 1235.797, 1245.591, 1248.513, 1269.18, 1296.231, 1298.746, 1300.536, 1332.55, 1346.319, 1359.022, 1361.958, 1363.577, 1367.157, 1368.197, 1400.488, 1403.028, 1410.137, 1412.203, 1431.278, 1469.979, 1480.031, 1486.336, 1491.72, 1519.956, 1524.682, 1531.001, 1533.87, 1536.279, 1538.293, 1543.612, 1546.231, 1574.48, 1579.311, 1582.497, 1621.001, 1643.755, 1659.939, 1663.98, 1671.733, 1685.674, 1800.66, 1802.134, 1850.214, 1862.417, 1879.148, 1928.648, 1930.533, 1957.27, 1960.728, 1986.567, 2016.909, 2081.118, 2108.262, 2139.712, 2177.437, 

In [ ]:
TimePoint = 0+5;cnt = 0
time_label={};resolution = 1
for names in channelList:
    time_label[names]=[]
while raw_filter.last_samp/1000 - TimePoint > 5:
    if any(abs(np.array(spindles) - TimePoint) < 1):
        print(TimePoint,np.array(spindles)[np.argmin(abs(np.array(spindles)-TimePoint))])
    else:
        print(TimePoint)
    for ii,names in enumerate(channelList):
        if Threshold_test(TimePoint,raw_alpha,raw_spindle,raw_muscle,ii):
            tempSegment,timeSpan=cut_segments(raw_filter,TimePoint,ii,windowsize = 1.5)
            normalziedSegment = normalize(tempSegment[0,:3000])#key step!!!
            predictedLabel = clf.predict(normalziedSegment)
            if predictedLabel == 1:
                # only find the peak of spindles, if not, why bother
                peakXval = eegPinelineDesign.CenterAtPeakOfWindow(TimePoint,1.5,raw_filter,ii)
                time_label[names].append([[TimePoint-1.5,TimePoint+1.5],peakXval,predictedLabel])
                print([TimePoint-1.5,TimePoint+1.5],peakXval,predictedLabel)
            else:
                time_label[names].append([[TimePoint-1.5, TimePoint+1.5],TimePoint,predictedLabel])
                print([TimePoint-1.5,TimePoint+1.5],TimePoint,predictedLabel)
        else:
            time_label[names].append([[TimePoint-1.5, TimePoint+1.5],TimePoint,0])
            print([TimePoint-1.5,TimePoint+1.5],TimePoint,0)
    TimePoint += resolution

5
[3.5, 6.5] 5 0
[3.5, 6.5] 5 0
[3.5, 6.5] 5 0
[3.5, 6.5] 5 [0]
[3.5, 6.5] 5 [0]
[3.5, 6.5] 5 0
6
[4.5, 7.5] 6 0
[4.5, 7.5] 6 [0]
[4.5, 7.5] 6 0
[4.5, 7.5] 6 [0]
[4.5, 7.5] 6 [0]
[4.5, 7.5] 6 0
7
[5.5, 8.5] 7 0
[5.5, 8.5] 7 0
[5.5, 8.5] 7 0
[5.5, 8.5] 7 [0]
[5.5, 8.5] 7 [0]
[5.5, 8.5] 7 0
8
[6.5, 9.5] 8 0
[6.5, 9.5] 8 [0]
[6.5, 9.5] 8 0
[6.5, 9.5] 8 [0]
[6.5, 9.5] 8 [0]
[6.5, 9.5] 8 0
9
[7.5, 10.5] 9 0
[7.5, 10.5] 9 [0]
[7.5, 10.5] 9 0
[7.5, 10.5] 9 [0]
[7.5, 10.5] 10.459 [1]
[7.5, 10.5] 9 0
10
[8.5, 11.5] 10 0
[8.5, 11.5] 10 [0]
[8.5, 11.5] 10 0
[8.5, 11.5] 10 [0]
[8.5, 11.5] 10 [0]
[8.5, 11.5] 10 0
11
[9.5, 12.5] 11 0
[9.5, 12.5] 11 [0]
[9.5, 12.5] 11 0
[9.5, 12.5] 11 [0]
[9.5, 12.5] 11 [0]
[9.5, 12.5] 11 0
12
[10.5, 13.5] 12 0
[10.5, 13.5] 10.529 [1]
[10.5, 13.5] 12 0
